<a href="https://colab.research.google.com/github/tinemyumi/saude-mental-datasus/blob/main/codigos-python/download_e_merge_sihsus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Relatório**

**Integrantes**
- Larissa Yumi Tinem
- Leo Tsuchida Hoçoya

**Conteúdo**
- Código para download dos dados do datasus, concatenação dos arquivos .parquet em meses para um arquivo ano.parquet e transferência permamente desses dados para o Google Drive.

**Histórico de alterações:**
- **13/09/2025** - Larissa Tinem - Criação do arquivo e importação dos dados do SIH-SUS.
- **20/09/2025** - Larissa Tinem - Obtenção dos dados do IBGE usando a API do IBGE via pysus (IBGE.FetchData)
- **08/10/2025** - Larissa Tinem - Download do restante dos dados do SIH-SUS (2014 a 2024) e concatenação dos arquivos (2022 a 2024)
- **16/10/2025** - Larissa Tinem - Concatenação do restante dos arquivos (2014 a 2021)



# **Conexão com o Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Baixando dados do Datasus (SIH-SUS) e IBGE via pysus**

Baixando dados do Datasus(SIH-SUS, Estado de São Paulo, no ano de 2015) através da biblioteca pysus.

In [ ]:
# Instalação da biblioteca pysus
!pip install pysus

In [ ]:
# Código utilizado para realizar o download da base da de dados do datasus, a respeito do SIH-SUS

from pysus import SIH
import os
import pandas as pd

# 1. Carregando a biblioteca PySUS, os arquivos do datasus
sih = SIH().load()

# 2. Definição dos anos para realizar download (como exemplo o ano de 2015, mas pode ser uma lista dos anos desejáveis)
anos_para_baixar = [2024]
diretorio_base = '/content/dados_sih/'

# 3. Loop para processar cada ano
for ano in anos_para_baixar:
    diretorio_ano = os.path.join(diretorio_base, str(ano))
    os.makedirs(diretorio_ano, exist_ok=True)

    print(f"Buscando e baixando dados para o ano: {ano}...")

    # Buscando os arquivos para o ano atual e para todos os meses
    arquivos_ano = sih.get_files("RD", uf="sp", year=ano, month=list(range(1, 13)))

    # Baixando os arquivos para a pasta específica do ano
    sih.download(arquivos_ano, local_dir=diretorio_ano)

    print(f"Download para {ano} concluído na pasta: {diretorio_ano}")

print("\nProcesso de download para todos os anos concluído!")

Buscando e baixando dados para o ano: 2024...


RDSP2412.parquet: 100%|██████████| 792k/792k [00:46<00:00, 16.9kB/s]

Download para 2024 concluído na pasta: /content/dados_sih/2024

Processo de download para todos os anos concluído!


In [ ]:
# Código de importação dados do content para o Google Drive

from google.colab import drive
import shutil

# Monta o seu Google Drive no Colab
drive.mount('/content/drive')

# Caminho da pasta de origem no Colab
caminho_origem = '/content/dados_sih/'

# Caminho de destino Google Drive
# A pasta principal do seu Drive é '/content/drive/MyDrive/'
caminho_destino = '/content/drive/MyDrive/DadoDataSUS/dados_sih/'

print("Iniciando a transferência dos arquivos para o Google Drive...")

try:
    shutil.move(caminho_origem, caminho_destino)
    print("\nTransferência concluída com sucesso!")
    print(f"Os arquivos agora estão salvos em: {caminho_destino}")

except FileNotFoundError:
    print("\nErro: A pasta de origem não foi encontrada. Verifique se o caminho está correto.")

except Exception as e:
    print(f"\nOcorreu um erro durante a transferência: {e}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Iniciando a transferência dos arquivos para o Google Drive...

Transferência concluída com sucesso!
Os arquivos agora estão salvos em: /content/drive/MyDrive/DadoDataSUS/dados_sih/


# **Unir meses do ano em único .parquet**

Este código ealiza a leitura de todos os arquivos .parquet mensais de um ano específico, concatena os dados em um único DataFrame e exporta o resultado consolidado em um único arquivo .parquet, facilitando a análise e o gerenciamento das informações

Data de realização: 08/10/2025 Autor: Larissa Tinem

In [ ]:
!pip install fastparquet
!pip install pyarrow

In [ ]:
import os
import pandas as pd

#Escolha do ano
ano = 2021

# Caminho base
caminho_base = "/content/drive/MyDrive/DadoDataSUS/dados_sih/dados_datasus"
pasta_ano = os.path.join(caminho_base, str(ano))

# Caminho destino
caminho_destino = "/content/drive/MyDrive/DadoDataSUS/dados_sih/dados_datasus"
os.makedirs(caminho_destino, exist_ok=True)

# Lista todos os arquivos parquet do ano
arquivos = sorted([os.path.join(pasta_ano, f) for f in os.listdir(pasta_ano) if f.endswith(".parquet")])

print(f"🔄 Concatenando {len(arquivos)} arquivos do ano {ano}...")

# Concatena todos os meses
dfs = [pd.read_parquet(arq) for arq in arquivos]
df_anual = pd.concat(dfs, ignore_index=True)

# Salva o arquivo consolidado
destino = os.path.join(caminho_destino, f"{ano}.parquet")
df_anual.to_parquet(destino, index=False)

print(f"✅ Arquivo do ano {ano} salvo em {destino}")


🔄 Concatenando 12 arquivos do ano 2021...
✅ Arquivo do ano 2021 salvo em /content/drive/MyDrive/DadoDataSUS/dados_sih/dados_datasus/2021.parquet


In [ ]:
import os
import pandas as pd
import gc

# ======================
# CONFIGURAÇÕES
# ======================
ano = 2024
caminho_base = '/content/drive/MyDrive/DadosSIA'
pasta_ano = os.path.join(caminho_base, str(ano))
caminho_destino = '/content/drive/MyDrive/DadosSIA/Concatenado'
os.makedirs(caminho_destino, exist_ok=True)

# Lista de arquivos parquet
arquivos = sorted([os.path.join(pasta_ano, f) for f in os.listdir(pasta_ano) if f.endswith(".parquet")])
print(f"🔍 {len(arquivos)} arquivos encontrados para o ano {ano}\n")

# ======================
# LEITURA SEGURA
# ======================
dfs = []
arquivos_ok = []
arquivos_ruins = []

for arq in arquivos:
    try:
        df_temp = pd.read_parquet(arq, engine='fastparquet')  # força engine alternativa
        dfs.append(df_temp)
        arquivos_ok.append(os.path.basename(arq))
        print(f"✅ OK: {os.path.basename(arq)} ({len(df_temp)} linhas)")
    except Exception as e:
        arquivos_ruins.append((os.path.basename(arq), str(e)))
        print(f"❌ ERRO: {os.path.basename(arq)} → {type(e).__name__}")

    # limpeza de memória
    del df_temp
    gc.collect()

# ======================
# CONCATENA E SALVA
# ======================
if dfs:
    df_anual = pd.concat(dfs, ignore_index=True)
    destino = os.path.join(caminho_destino, f"{ano}.parquet")
    df_anual.to_parquet(destino, index=False)
    print(f"\n✅ Arquivo final salvo em: {destino}")
    print(f"📊 Total de registros: {len(df_anual)}")
else:
    print("\n⚠️ Nenhum arquivo válido foi carregado!")

# ======================
# RELATÓRIO FINAL
# ======================
print("\n--- RESUMO ---")
print(f"✅ Arquivos OK: {len(arquivos_ok)}")
print(f"❌ Arquivos com erro: {len(arquivos_ruins)}")

if arquivos_ruins:
    print("\nArquivos problemáticos:")
    for nome, erro in arquivos_ruins:
        print(f" - {nome}: {erro[:120]}...")


🔍 12 arquivos encontrados para o ano 2024

✅ OK: part-01.parquet (375662 linhas)
✅ OK: part-02.parquet (374132 linhas)
✅ OK: part-03.parquet (407639 linhas)
✅ OK: part-04.parquet (415240 linhas)
✅ OK: part-05.parquet (401785 linhas)
✅ OK: part-06.parquet (415727 linhas)
✅ OK: part-07.parquet (426559 linhas)
✅ OK: part-08.parquet (449685 linhas)
✅ OK: part-09.parquet (459183 linhas)
✅ OK: part-10.parquet (471874 linhas)
✅ OK: part-11.parquet (418281 linhas)
✅ OK: part-12.parquet (358918 linhas)

✅ Arquivo final salvo em: /content/drive/MyDrive/DadosSIA/Concatenado/2024.parquet
📊 Total de registros: 4974685

--- RESUMO ---
✅ Arquivos OK: 12
❌ Arquivos com erro: 0


In [ ]:
# Salvando os dados como parquet no Google Drive

from google.colab import drive
drive.mount('/content/drive')

# Salvando como parquet
ds.to_dataframe().to_parquet('/content/drive/MyDrive/DadoDataSUS/dados_ibge/populacao_residente_2015.parquet', index=False)
ds_sp.to_dataframe().to_parquet('/content/drive/MyDrive/DadoDataSUS/dados_ibge/populacao_residente_municipio_2015.parquet', index=False)
print("Salvo com sucesso")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Salvo com sucesso
